In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from scipy.signal import find_peaks
import scipy.signal.windows as win
import scipy as sp
import warnings
import re
from ipywidgets import interact,widgets
from scipy.optimize import curve_fit
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc
from scipy.misc import derivative
from matplotlib.colors import hsv_to_rgb
import warnings
from functions_master import *
from glob import glob
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.gridspec as gridspec

# Reference.

9e-10 

In [13]:

# Suprimir FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)


c = 0.299792458 # speed of light mm/ps

def FourierT2(f,N):
    return np.conj(sp.fft.fft(f.values,n=N))

def calcular_n(omega, phi_exp, c, d):

    return 1 + (c / (omega * d)) * phi_exp

def calcular_kappa(omega, T_exp, n, c, d):

    term = ((n + 1)**2 / (4 * n)) * T_exp
    return -(c / (omega * d)) * np.log(term)


def extraer_angulos(nombres_archivos):
    """
    Extrae el patrón que está después del último '__' y antes de '.dat'
    Ejemplo: '...__hori-80deg.dat' → 'hori-80deg'
    """
    patrones = []
    patron_regex = re.compile(r'__([^__]+)\.dat$')  # Captura lo entre último __ y .dat
    
    for nombre in nombres_archivos:
        coincidencia = patron_regex.search(nombre)
        if coincidencia:
            patrones.append(coincidencia.group(1))
    
    return patrones


In [23]:
# Configuración desde el archivo txt (simulada aquí como un diccionario)
config = {
    'figure.constrained_layout.use': True,
    'figure.figsize': (5.4, 3.2),
    'figure.titlesize': 'small',
    'figure.labelsize': 'small',
    'font.size': 10,
    'font.family': 'serif',
    'text.usetex': False,
    'pgf.texsystem': 'lualatex',
    'pgf.rcfonts': False,
    'pgf.preamble': r'\usepackage{siunitx,fontspec}\setmainfont{Times New Roman}',
    'lines.linewidth': 1,
    'lines.markerfacecolor': 'none',
    'lines.markersize': 4,
    'legend.frameon': False,
    'legend.markerscale': 0.8,
    'legend.fontsize': 'small',
    'legend.title_fontsize': 'small',
    'legend.handlelength': 1.5,
    'legend.handletextpad': 0.2,
    'xtick.top': True,
    'xtick.major.size': 5,
    'xtick.major.width': 0.8,
    'xtick.minor.width': 0.8,
    'xtick.minor.size': 3,
    'xtick.labelsize': 'small',
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.major.size': 5,
    'ytick.major.width': 0.8,
    'ytick.minor.width': 0.8,
    'ytick.minor.size': 3,
    'ytick.labelsize': 'small',
    'ytick.direction': 'in',
    'axes.linewidth': 0.8,
    'axes.axisbelow': False,
    'axes.labelsize': 'medium',
    'axes.labelpad': 2,
    'figure.constrained_layout.h_pad': 0.02,
    'figure.constrained_layout.w_pad': 0.01,
    'figure.constrained_layout.hspace': 0.02,
    'figure.constrained_layout.wspace': 0.02,
    'svg.fonttype': 'none'
}

# Aplicar la configuración
#plt.rcParams.update(config)#

In [29]:
ruta_actual = os.getcwd()  # Muestra la ruta actual

In [30]:
ruta_actual

'C:\\Users\\LAB103\\Documents\\THz\\THzEsteban\\analisis_datos_rapido'

In [31]:
ruta_actual = os.getcwd()

sample1_ang_csv = os.path.join(ruta_actual,"sample1_ang.csv")
sample2_ang_csv= os.path.join(ruta_actual,"sample2_ang.csv")

carpeta_sample1 = os.path.join(ruta_actual,"src\\sample1_ang")
carpeta_ref_sample1 = os.path.join(ruta_actual,"src\\reference1")

sample1_med = glob(os.path.join(carpeta_sample1, "*"))
angulos_sample1  = extraer_angulos(sample1_med)
rutas_ref_sample1 = glob(os.path.join(carpeta_ref_sample1, "*"))



# FULL DATA

In [32]:
import matplotlib.gridspec as gridspec

In [33]:
def anim2(left,right_sample,right_subs,index,desv,d):
    
    right_ref= right_subs
    left = left
    N = 2**15

    f = plt.figure(figsize=(7, 7), dpi=200)

# Configuración de gridspec: 2 filas, 2 columnas, con alturas relativas 3:1
    gs = gridspec.GridSpec(2, 2, height_ratios=[3, 1], width_ratios=[1, 1])

    # --- Primera fila (más grande) ---
    ax1 = plt.subplot(gs[0, 0])  # Subplot 221
    ax2 = plt.subplot(gs[0, 1])  # Subplot 222

    # --- Segunda fila (más pequeña) ---
    ax3 = plt.subplot(gs[1, 0])  # Subplot 223
    ax4 = plt.subplot(gs[1, 1])  # Subplot 224

    print(index)
    path_signal = sample1_med[int(index)]
    path_ref = rutas_ref_sample1[0]
    j = round(index)
    print(j)

    num_files = len(sample1_med)
    angles = np.linspace(0, 360, num_files, endpoint=False)  # ángulos de 0 a 350

    # Usar un mapa de colores cíclico
    colors = [hsv_to_rgb([angle / 360.0, 0.6, 0.85]) for angle in angles]

    params_window1 = ['gaussian', desv] #['nuttall']#['flattop'] #['gaussian', desv]#['nuttall'] #['kaiser',desv] #['flattop']# ['gaussian', desv]


    phase, y_signal_ventaneada, y_substrate_padding, ventana = getSignalWindowed(path_signal,
                                                                                 path_ref,
                                                                                 left,
                                                                                 right_sample,
                                                                                 right_subs,
                                                                                 params_window1)

    y_subs_ventana = pd.Series(y_substrate_padding*ventana)

    y_signal_ventaneada = pd.Series(y_signal_ventaneada*ventana)


    val = 2
    
    ax1.plot(y_subs_ventana/max(y_subs_ventana), 'k',label='Ref')
    ax1.plot(ventana/max(ventana), 'k--')
    ax1.set_title("Reference")

    ax1.plot(y_signal_ventaneada/max(y_signal_ventaneada) + val,label='Sam')
    ax1.plot(ventana/max(ventana) + val, 'k--')
    ax1.legend(loc='lower left')
        

    right_sample = right_sample


    # params_window1 = ['flattop'] #['gaussian', desv]
    k = 15
    nu = sp.fft.fftfreq(N, 1/30)
        # Aplicar ventana de Hamming a la señal
    
    fft_y_signal_ventaneada = FourierT2(y_signal_ventaneada,N)[1:len(nu)//k]
    fft_y_subs_ventaneada = FourierT2(y_subs_ventana,N)[1:len(nu)//k]

    # fft_window = FourierT2(pd.Series(ventana),N)[1:len(nu)//k]

    # Filtrar las frecuencias
    nu = nu[1:len(nu)//k]
    xmin, xmax = 0.15, 1.0
    mask = (nu >= xmin) & (nu <= xmax)

    fft_y_signal_ventaneada = fft_y_signal_ventaneada[mask]
    fft_y_subs_ventaneada = fft_y_subs_ventaneada[mask]
    nu_filtradas = nu[mask]

    T = fft_y_signal_ventaneada/fft_y_subs_ventaneada*np.exp(1j*2*np.pi*nu_filtradas*phase*1/30)
    d = d
    
    T_exp = abs(T)

    phi_exp = np.angle(T)

    phi_exp = np.unwrap(phi_exp)

    # plt.plot(nu_filtradas,abs(T)/max(abs(T)))
    # plt.yscale('log')


    ax2.plot(nu_filtradas, abs(T)/max(abs(T)))
    ax2.set_ylabel(r"$T_{exp}$")
    ax2.set_yscale('log')

  

    # =============================================
    # Gráficos 223 y 224 (segunda fila)
    # =============================================
    # Gráfico 223 (ax3)
    n = calcular_n(nu_filtradas*2*np.pi, phi_exp, c, d)
    ax3.plot(nu_filtradas, n,'k')
    ax3.set_xlabel(r"$\nu$ (THz)")
    ax3.set_ylabel(r"$n$")

    # Gráfico 224 (ax4)
    ax4.set_ylabel(r"$\kappa$")
    # ax4.set_title(angulos_sample1[j])
    ax4.plot(nu_filtradas, calcular_kappa(nu_filtradas*2*np.pi, T_exp, n, c, d),'k')
    ax4.set_xlabel(r"$\nu$ (THz)")

# Ajuste final
# plt.tight_layout()



    # plt.legend(bbox_to_anchor=(1.05, 3.0), loc='upper left', title="Ángulo de polarización")
    plt.tight_layout()

opc=dict(continuous_update=False,readout_format=".3f")
interact(anim2,       
        left= widgets.FloatSlider(min=40,max=70,value=57.5,step=0.1,**opc),
        right_sample= widgets.FloatSlider(min=50,max=70,value=63.5,step=0.1,**opc),
        right_subs= widgets.FloatSlider(min=50,max=70,value=63.5,step=0.1,**opc),
        index= widgets.FloatSlider(min=0,max=len(sample1_med)-1,value=0,step=1,**opc),
        desv = widgets.FloatSlider(min=0,max=1000,value=150,step=0.001,**opc),
        d = widgets.FloatSlider(min=0,max=1,value=0.627,step=0.01,**opc)

        )

interactive(children=(FloatSlider(value=57.5, continuous_update=False, description='left', max=70.0, min=40.0,…

<function __main__.anim2(left, right_sample, right_subs, index, desv, d)>

In [11]:
len(sample1_med)

1

In [ ]:
%conda update numpy

